In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v0.interface import vlstm_fwbw_torch_obw, vlstm_fwbw_cuda

from src.vlstm_fwbw_v0.interface import vlstm_fw_torch, vlstm_fw_cuda
from src.vlstm_fwbw_v0.interface import vlstm_bw_torch_obw, vlstm_bw_cuda
from src.vlstm_fwbw_v0.torch_impl import vlstm_fw_tiled_torch

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v0/build.ninja...
Building extension module vlstm_fwbw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/nvcc --generate-dependencies-with-compile --dependency-output kernel_bw.cuda.o.d -ccbin /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_fwbw_v0 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERS

Loading extension module vlstm_fwbw_v0...


## CUDA vLSTM backward

In [3]:
S = 16 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# random:
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

dHs = qs.clone()
qs, qs.shape, len(qs.view(-1))

(tensor([[[[-9.2466e-01, -4.2534e-01, -2.6438e+00,  1.4518e-01, -1.2087e-01, -5.7973e-01, -6.2285e-01, -3.2839e-01],
           [-1.0745e+00, -3.6314e-01, -1.6711e+00,  2.2655e+00,  3.1168e-01, -1.8419e-01,  1.2866e+00,  1.1820e+00],
           [-1.2706e-01,  1.2169e+00,  1.4353e+00,  1.0605e+00, -4.9413e-01, -1.4244e+00, -7.2443e-01, -1.2973e+00],
           [ 6.9690e-02, -7.4066e-03,  1.8969e+00,  6.8778e-01, -7.7948e-02, -8.3728e-01,  1.3506e+00, -2.8792e-01],
           [-5.9653e-01, -3.2826e-01, -9.0860e-01, -8.0594e-01, -7.4067e-01, -5.0385e-02,  5.4348e-01,  1.5150e+00],
           [ 1.4121e-02,  4.5320e-01,  1.6349e+00,  7.1239e-01, -1.8057e-01,  1.0252e+00, -1.4622e+00, -7.5538e-01],
           [-1.8364e-01,  3.8239e-01,  3.9177e-01, -8.2991e-02,  8.9712e-01, -1.1123e+00,  1.1165e-01,  4.8628e-01],
           [-5.4994e-01, -3.2309e-01, -5.4689e-01,  9.0488e-01,  2.8369e-01,  1.2103e-01,  4.7297e-01, -1.0823e+00],
           [-3.3446e-02, -9.7344e-01,  9.5592e-01, -1.1795e+00, 

### bw kernel match direct call

In [5]:
hs_pt, n_pt, m_pt, _, matD_pt = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)

In [6]:
dQs_pt, dKs_pt, dVs_pt, dIgs_pt, dFgs_pt, delta_D_pt, delta_Dtilde_pt, delta_fbar_pt, mat_P_pt, mat_C_pt = vlstm_bw_torch_obw(
    delta_Htilde=dHs,
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
    var_n=n_pt,
    var_m=m_pt,
)

In [7]:
def rev_cumsum(x, dim=-1):
    return x.flip(dims=(dim,)).cumsum(dim).flip(dims=(dim,))

In [8]:
# calculate fgrads trick
df = (dQs_pt * qs - dKs_pt * ks).sum(-1, keepdim=True)
df = rev_cumsum(df, dim=-2)
df_pt = df * torch.nn.functional.sigmoid(-fgs)
# df_pt

In [9]:
# df_pt - dFgs_pt

In [10]:
hs_cu, n_cu, m_cu, _ = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs, fgate_preact=fgs)

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5664
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [11]:
dQs_cu, dKs_cu, dVs_cu, dIgs_cu, dFgs_cu, matC_cu, deltaDcsChunkArr_cu, deltaDcsVec_cu = vlstm_bw_cuda(
    delta_Htilde=dHs,
    mat_Q=qs,
    mat_K=ks,
    mat_V=vs,
    igate_preact=igs,
    fgate_preact=fgs,
    n=n_pt,
    m=m_pt,
)

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 8
blocksxy: 1-1, threadsxy: 4-4, shared_mem in bytes: 7648
In BW-Kernel: gdim.x: 1, gdim.y: 1, gdim.z: 1, bdim.x: 4, bdim.y: 4
In BW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [15]:
RTOL = 1e-10
ATOL = 4e-4
print(f"delta Q match: {torch.allclose(dQs_cu, dQs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta K match: {torch.allclose(dKs_cu, dKs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta V match: {torch.allclose(dVs_cu, dVs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta Igate match: {torch.allclose(dIgs_cu.unsqueeze(-1), dIgs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta Fgate match: {torch.allclose(dFgs_cu.unsqueeze(-1), dFgs_pt, rtol=RTOL, atol=ATOL)}")

delta Q match: True
delta K match: False
delta V match: False
delta Igate match: True
delta Fgate match: True


In [13]:
torch.abs(matC_cu - mat_P_pt).max()

tensor(1.0967e-05, device='cuda:0')

In [14]:
mat_P_pt

tensor([[[[-1.2881e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00,
            0.0000e+00, -0.0000e+00],
          [ 5.7866e-01, -7.4848e-02,  0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-1.7459e+00,  4.6757e-01, -5.1388e-01, -0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 9.8656e-02,  2.1464e-01, -4.3427e-01,  3.4224e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 9.4020e-02, -6.7717e-02,  6.0971e-02,  1.0002e+00, -1.1657e+00, 

In [13]:
dIgs_pt

tensor([[[[0.0049],
          [0.0216],
          [0.0030],
          [0.1189],
          [0.0950],
          [0.0626],
          [0.0695],
          [0.1783],
          [0.0826],
          [0.1365],
          [0.0476],
          [2.2342],
          [0.3448],
          [0.2026],
          [0.9057],
          [0.5566]]]], device='cuda:0')

In [51]:
dIgs_cu.unsqueeze(-1)

tensor([[[[0.0049],
          [0.0216],
          [0.0030],
          [0.1189],
          [0.0950],
          [0.0626],
          [0.0695],
          [0.1783],
          [0.0826],
          [0.1365],
          [0.0476],
          [2.2342],
          [0.3448],
          [0.2026],
          [0.9057],
          [0.5566]]]], device='cuda:0')

In [52]:
dIgs_cu.unsqueeze(-1) - dIgs_pt

tensor([[[[2.3283e-09],
          [1.1176e-08],
          [1.8626e-09],
          [8.9407e-08],
          [4.4703e-08],
          [2.9802e-08],
          [1.4901e-08],
          [5.9605e-08],
          [3.7253e-08],
          [1.0431e-07],
          [5.2154e-08],
          [6.4373e-06],
          [6.2585e-07],
          [3.8743e-07],
          [1.6689e-06],
          [7.7486e-07]]]], device='cuda:0')

In [53]:
dFgs_pt - dFgs_cu.unsqueeze(-1)

tensor([[[[ 0.0000e+00],
          [-1.6298e-09],
          [-3.7253e-09],
          [-1.3970e-08],
          [-1.4901e-08],
          [-2.2352e-08],
          [-2.9802e-08],
          [-3.7253e-08],
          [-2.2352e-08],
          [-2.9802e-08],
          [-9.6858e-08],
          [-9.6858e-08],
          [-1.3709e-06],
          [-7.7486e-07],
          [-3.5763e-07],
          [-2.8312e-07]]]], device='cuda:0')

In [54]:
dFgs_cu.unsqueeze(-1)

tensor([[[[0.0000],
          [0.0030],
          [0.0068],
          [0.0145],
          [0.0235],
          [0.0460],
          [0.0629],
          [0.0788],
          [0.0599],
          [0.0466],
          [0.0744],
          [0.0331],
          [0.5846],
          [0.4395],
          [0.2993],
          [0.2302]]]], device='cuda:0')

In [55]:
dFgs_pt

tensor([[[[0.0000],
          [0.0030],
          [0.0068],
          [0.0145],
          [0.0235],
          [0.0460],
          [0.0629],
          [0.0788],
          [0.0599],
          [0.0466],
          [0.0744],
          [0.0330],
          [0.5846],
          [0.4395],
          [0.2993],
          [0.2302]]]], device='cuda:0')

In [56]:
(matC_cu - delta_Dtilde_pt.cumsum(-1).tril(-1))[:, :, :, -10:]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [1.8626e-08, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

In [18]:
dQs_cu - dQs_pt, torch.abs(dQs_cu - dQs_pt).max()

(tensor([[[[-5.9605e-08,  1.7881e-07, -2.9802e-07,  2.6822e-07, -5.3644e-07, -2.9802e-07, -4.0978e-08,  2.9802e-07],
           [ 1.3970e-08, -8.9407e-08,  1.3411e-07, -1.0431e-07,  1.7881e-07,  1.0431e-07,  1.8626e-08, -1.1921e-07],
           [ 1.4901e-08,  0.0000e+00, -1.1921e-07,  4.4703e-08, -2.9802e-07, -1.4901e-07, -8.9407e-08,  1.7881e-07],
           [ 5.2154e-08, -2.0862e-07,  1.1921e-07, -2.0862e-07,  5.2154e-08,  1.1921e-07,  5.9605e-08,  5.5879e-09],
           [-3.5390e-08, -7.0781e-08, -4.1723e-07, -1.0431e-07,  2.9802e-07,  1.6391e-07,  5.9605e-08, -3.5763e-07],
           [-8.0466e-07, -7.4506e-08,  1.5646e-07, -2.6822e-07,  3.0994e-06, -6.6357e-09, -7.2643e-08,  8.0466e-07],
           [-3.5018e-07,  2.2352e-08, -5.9605e-07, -5.6624e-07,  4.5300e-06, -7.7486e-07, -1.6391e-07,  1.3113e-06],
           [ 2.1458e-06,  8.9407e-07, -3.0994e-06,  2.9802e-07,  2.2650e-06, -2.5034e-06, -1.6689e-06,  2.8871e-08],
           [-1.9073e-06,  8.3447e-07,  5.9605e-07,  2.0862e-07, 

In [19]:
dKs_cu - dKs_pt, torch.abs(dKs_cu - dKs_pt).max()

(tensor([[[[ 1.7881e-07, -2.9802e-07, -8.3489e-01, -1.1921e-07,  1.7881e-07,  3.5763e-07,  8.3447e-07,  7.1526e-07],
           [ 1.3039e-08,  7.4506e-08,  2.0862e-07,  4.4703e-08, -8.1536e-01, -8.9407e-08, -1.1176e-08, -1.1921e-07],
           [-5.9081e-09, -1.0431e-07, -3.5763e-07, -1.4901e-07,  5.2154e-08,  1.4901e-07, -4.4421e-01,  2.3842e-07],
           [-7.4506e-08, -9.6858e-08, -3.8184e-08, -5.3644e-07, -1.0431e-07, -4.4703e-07,  2.9802e-07,  7.1526e-07],
           [ 1.1921e-07, -4.2789e-01,  3.2783e-07,  3.5763e-07,  1.7881e-07,  1.0431e-07, -2.0862e-07, -3.5763e-07],
           [-3.7253e-07,  1.3709e-06,  3.2187e-06, -8.4603e-01,  1.4305e-06, -2.3842e-07, -1.9073e-06, -5.5134e-07],
           [ 2.5332e-07, -1.7881e-07,  2.2352e-07, -4.7684e-07, -5.9605e-07, -1.3354e+00, -9.6858e-08,  1.3039e-07],
           [ 9.5367e-07,  2.0862e-07,  1.6689e-06, -2.3842e-06, -7.1526e-07, -2.9802e-07, -5.9605e-07, -4.2066e-01],
           [-7.4506e-08,  1.4305e-06, -3.8147e-06,  9.5367e-07, 

In [20]:
dVs_cu - dVs_pt, torch.abs(dVs_cu - dVs_pt).max()

(tensor([[[[       -inf,  4.7684e-07,  2.8610e-06, -9.5367e-07,  2.0489e-08,  9.5367e-07, -2.3842e-07, -5.3644e-07],
           [ 5.9605e-08,  3.7253e-09,        -inf, -1.1921e-07,  0.0000e+00,  4.4703e-08, -8.9407e-08, -2.4214e-08],
           [-7.4506e-09,  2.3842e-07,  2.3842e-07,  2.3842e-07,        -inf, -2.3842e-07, -1.1921e-07, -3.5763e-07],
           [-5.9605e-08, -2.3842e-07,  1.0729e-06,  1.4901e-07, -1.1921e-07, -9.5367e-07,        -inf,  1.2666e-07],
           [-4.4703e-08, -3.7253e-08,  1.6764e-07, -1.7881e-07, -1.0431e-07, -4.4703e-08,  0.0000e+00,  1.7881e-07],
           [-4.7684e-07, -7.6306e-02, -1.6689e-06, -8.0466e-07,  1.6689e-06, -2.7418e-06,  1.7285e-06,  1.4901e-06],
           [-1.2666e-07, -1.3113e-06,  2.9802e-08,  3.0957e-01, -1.3113e-06,  1.5497e-06,  7.1526e-07, -1.6689e-06],
           [ 1.1176e-07, -6.5565e-07,  1.3113e-06, -1.1921e-06, -4.1723e-07,  7.2678e-01,  2.6822e-07,  1.1921e-07],
           [ 5.9605e-08,  1.5087e-07, -2.3842e-07, -1.3411e-07, 

In [52]:
dIgs_cu - dIgs_pt.squeeze(-1), dIgs_cu.shape

(tensor([[[ 6.1219e+00,  7.3210e-01,  2.4523e+00,  3.0649e+00,  2.1296e+00,  7.6970e-01,  8.0635e+00,  2.6605e-01,  2.8676e+00, -1.7003e+00,  2.1132e-01,  1.3613e+00,  4.2462e-01,  8.0890e+00,
            5.1872e+00,  7.0967e-01,  1.3349e+01, -9.7979e-01,  3.5690e+00, -5.1287e-01,  6.9646e-01,  1.0673e+00,  5.5749e+00, -3.3935e-01,  8.6814e-05,  3.3688e+00,  1.1465e-01,  1.2180e+00,
            5.3781e+00,  7.3750e+00,  1.0271e+00,  7.7895e+00,  3.4989e+00,  6.9227e-01,  1.2521e+01,  2.2172e-01,  4.7920e+00,  6.0415e-01,  2.8018e+00,  2.2682e-01,  1.5406e+00,  7.2706e+00,
            1.1437e+00,  3.1559e-01,  1.8419e+00,  4.0405e-02, -9.7288e-02,  9.9358e-01,  2.2841e+00, -6.4965e-01,  1.4312e+00,  2.5603e+00,  4.0268e+01,  1.7432e-01,  9.4742e+00,  2.1577e+00,
            3.5761e-02, -2.5396e-01,  7.6760e-01,  1.9595e+01,  4.0970e+00,  7.8107e-01,  2.4614e-02, -2.1997e+00]]], device='cuda:0'),
 torch.Size([1, 1, 64]))

In [22]:
dFgs_cu - delta_fbar_pt.squeeze(-1)

tensor([[[ 0.0000e+00, -1.7480e-03, -1.7160e-02, -6.9439e-03, -9.3492e-02, -1.0969e-01, -7.1794e-02, -3.0862e-02, -1.3550e-01, -1.1357e-01, -2.7724e-02, -2.4580e-02, -1.0732e+00, -7.8491e-01,
          -2.2732e-01, -5.2500e-01, -6.7238e-01, -9.5194e-01, -3.9267e-01, -2.5783e+00, -2.1379e+00, -1.0131e+00, -9.2257e-01, -5.0708e-01, -2.6059e-01, -2.1348e-01, -5.0075e-01, -1.1851e+00,
          -2.4597e+00, -1.8238e+00, -1.2771e+00, -9.5950e-01]]], device='cuda:0')

In [23]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, 5:17]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [6.2363e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

In [24]:
deltaDcsVec_cu.unsqueeze(-1), deltaDcsVec_cu.shape

(tensor([[[[0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [7.7359e-01],
           [3.7578e-01],
           [1.2229e-01],
           [8.2258e-02],
           [7.6685e-02],
           [7.1804e-02],
           [4.7018e-02],
           [2.3921e-02],
           [1.1210e-02],
           [3.8283e-03],
           [6.5380e-04],
           [4.0062e-04],
           [3.2340e-04],
           [2.8763e-04],
           [1.2127e-04],
           [1.0430e-04]]]], device='cuda:0'),
 torch.Size([1, 1, 32]))

In [25]:
dFgs_cu - delta_fbar_pt.squeeze(-1)

tensor([[[ 0.0000e+00, -1.7480e-03, -1.7160e-02, -6.9439e-03, -9.3492e-02, -1.0969e-01, -7.1794e-02, -3.0862e-02, -1.3550e-01, -1.1357e-01, -2.7724e-02, -2.4580e-02, -1.0732e+00, -7.8491e-01,
          -2.2732e-01, -5.2500e-01, -6.7238e-01, -9.5194e-01, -3.9267e-01, -2.5783e+00, -2.1379e+00, -1.0131e+00, -9.2257e-01, -5.0708e-01, -2.6059e-01, -2.1348e-01, -5.0075e-01, -1.1851e+00,
          -2.4597e+00, -1.8238e+00, -1.2771e+00, -9.5950e-01]]], device='cuda:0')

In [26]:
matC_cu[:, :, :, 16:], matC_cu.shape

(tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 

In [25]:
deltaDcsChunkArr_cu

tensor([[[[0.3891, 0.1329, 0.0227, 0.0139, 0.0112, 0.0100, 0.0042, 0.0036],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]]], device='cuda:0')

In [26]:
deltaDcsVec_cu, deltaDcsVec_cu.shape

(tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 7.7359e-01, 3.7578e-01, 1.2229e-01, 8.2258e-02, 7.6685e-02, 7.1804e-02, 4.7018e-02, 2.3921e-02, 1.1210e-02, 3.8283e-03, 6.5380e-04, 4.0062e-04, 3.2340e-04, 2.8763e-04,
           1.2127e-04, 1.0430e-04]]], device='cuda:0'),
 torch.Size([1, 1, 32]))

In [27]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, :-1]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [4.3366e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [8.9254e-04, 4.9177e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+0

In [28]:
# matC_cu - mat_P_pt
# matC_cu - mat_C_pt

In [29]:
matC_cu[:, :, :, :-1]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [4.3366e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [8.9254e-04, 4.9177e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+0

In [30]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, :-1]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [4.3366e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [8.9254e-04, 4.9177e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+0

In [31]:
(delta_Dtilde_pt.cumsum(-1).tril(-1) - matC_cu)[:,:,:,:-2]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-2.0373e-10,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-2.3283e-10, -1.8626e-09,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00

In [32]:
dFgs_cu, delta_fbar_pt.squeeze(-1)

(tensor([[[ 0.0000,  0.0000, -0.5523,  0.9238, -0.7350,  1.3800,  0.8676,  0.1297,  0.0000,  0.0000,  0.8821, -1.0133, -0.3634,  0.5101,  0.4179, -0.6888,  0.0000,  0.0000,  1.7916, -0.8479,  1.6484,
            0.3976, -0.5237,  0.1986,  0.0000,  0.0000, -0.7760,  1.0086, -0.5316,  0.7116,  0.5386,  1.8525]]], device='cuda:0'),
 tensor([[[0.0000, 0.0048, 0.0240, 0.0214, 0.1170, 0.1558, 0.1349, 0.1099, 0.1957, 0.1606, 0.1041, 0.0599, 1.7177, 1.3017, 0.6800, 1.1257, 1.5903, 1.1106, 1.3094, 3.0743, 3.5745, 2.7235, 1.6789,
           1.0875, 0.6045, 0.6773, 0.6834, 3.2018, 3.6670, 2.8881, 1.4775, 1.6754]]], device='cuda:0'))

In [33]:
dFgs_cu - delta_fbar_pt.squeeze(-1)

tensor([[[ 0.0000, -0.0048, -0.5762,  0.9024, -0.8520,  1.2242,  0.7328,  0.0197, -0.1957, -0.1606,  0.7780, -1.0732, -2.0810, -0.7917, -0.2621, -1.8145, -1.5903, -1.1106,  0.4822, -3.9221, -1.9261,
          -2.3259, -2.2026, -0.8889, -0.6045, -0.6773, -1.4594, -2.1932, -4.1986, -2.1765, -0.9389,  0.1771]]], device='cuda:0')

In [34]:
deltaDcsVec_cu, deltaDcsVec_cu.shape

(tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 7.7359e-01, 3.7578e-01, 1.2229e-01, 8.2258e-02, 7.6685e-02, 7.1804e-02, 4.7018e-02, 2.3921e-02, 1.1210e-02, 3.8283e-03, 6.5380e-04, 4.0062e-04, 3.2340e-04, 2.8763e-04,
           1.2127e-04, 1.0430e-04]]], device='cuda:0'),
 torch.Size([1, 1, 32]))

### bw kernel match through autograd

In [35]:
fgs_pt = fgs.clone().detach().requires_grad_(True)
igs_pt = igs.clone().detach().requires_grad_(True)
qs_pt = qs.clone().detach().requires_grad_(True)
ks_pt = ks.clone().detach().requires_grad_(True)
vs_pt = vs.clone().detach().requires_grad_(True)

In [36]:
# pytorch version QKV product (To test if this is still correct after changes to the code.)
# at some point we have to compare to the vlstm_fw_torch version.
# rs = qs @ ks.transpose(-1, -2) @ vs
# rs, rs.shape

In [37]:
# rs_torch = (qs @ ks.transpose(-1, -2) * torch.tril(torch.ones((B, NH, S, S))).to(device=DEVICE, dtype=DTYPE)) @ vs
# rs_torch, rs_torch.shape

In [38]:
# pytorch version
hs_pt, n_pt, m_pt = vlstm_fwbw_torch_obw(
    queries=qs_pt,
    keys=ks_pt,
    values=vs_pt,
    igate_preact=igs_pt,
    fgate_preact=fgs_pt,
)
hs_pt, hs_pt.shape

(tensor([[[[3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05],
           [2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04],
           [3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04],
           [1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03],
           [1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03],
           [2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03],
           [2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03],
           [1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03],
           [2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03],
           [3.2163e

In [39]:
fgs_cu = fgs.clone().detach().requires_grad_(True)
igs_cu = igs.clone().detach().requires_grad_(True)
qs_cu = qs.clone().detach().requires_grad_(True)
ks_cu = ks.clone().detach().requires_grad_(True)
vs_cu = vs.clone().detach().requires_grad_(True)

In [40]:
# cuda kernel
hs_cu, n_cu, m_cu, matD_cu = vlstm_fwbw_cuda(mat_Q=qs_cu, mat_K=ks_cu, mat_V=vs_cu, igate_preact=igs_cu.squeeze(-1), fgate_preact=fgs_cu.squeeze(-1))
hs_cu, hs_cu.shape

before kernel dispatch - float32!
B: 1, NH: 1, S: 32, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5664
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
BIdx.y=0, FLTIdx=7: qTileIdx=0, fChunkAccIterIdx=0 (<1), fWCIdx=0, f_acc=-4.943604, fTileColLast=0.000000
STR: BIdx.y=1: qTileIdx=0, fWCIdx=0 (<1), flatThreadIdx=7: fT_acc_res=-10.706538, fTileCol_val=-10.706538, fTileColLastVal=0.000000
BIdx.y=0, FLTIdx=7: qTileIdx=0, fChunkAccIterIdx=1 (<1), fWCIdx=0, f_acc=-4.943604, fTileColLast=0.000000
BIdx.y=0, FLTIdx=7: qTileIdx=1, fChunkAccIterIdx=0 (<3), fWCIdx=0, f_acc=-5.259939, fTileColLast=-10.706538
STR: BIdx.y=1: qTileIdx=1, fWCIdx=0 (<1), flatThreadIdx=7: fT_acc_res=-20.829517, fTileCol_val=-20.829517, fTileColLastVal=-10.706538
BIdx.y=0, FLTIdx=7: qTileIdx=1, fChunkAccIterIdx=1 (<3), fWCIdx=0, f_acc=-5.259939, fTileColLast=-10.706538


(tensor([[[[3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05],
           [2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04],
           [3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04],
           [1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03],
           [1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03],
           [2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03],
           [2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03],
           [1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03],
           [2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03],
           [3.2163e

In [41]:
# forward match
hs_pt - hs_cu

tensor([[[[-1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11],
          [-1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10],
          [-1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10],
          [-1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09],
          [-1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09],
          [-9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10],
          [-9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10],
          [-1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09],
          [-2.3283e-10, -2.3283e-10, -2.3283e-10, -2.3283e-10, -2.3283e-

In [42]:
hs_pt.sum().backward()

ValueError: too many values to unpack (expected 5)

In [ ]:
qs_pt.grad, ks_pt.grad, vs_pt.grad, igs_pt.grad, fgs_pt.grad

(tensor([[[[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006],
           [0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011],
           [0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010],

In [ ]:
hs_cu.sum().backward()

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 7136
In BW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In BW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [ ]:
qs_cu.grad, ks_cu.grad, vs_cu.grad, igs_cu.grad, fgs_cu.grad

(tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.]]]], device='cuda:0'),
 tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.